In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json
from datetime import datetime
import os  # Dodaj ten import na początku pliku

def setup_driver():
    """Konfiguracja przeglądarki Chrome"""
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Tryb bezinterfejsowy
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--lang=pl')
    return webdriver.Chrome(options=options)

def search_apps_by_keyword(driver, keyword, max_apps=1000):
    """Wyszukuje aplikacje dla danego słowa kluczowego"""
    apps = []
    try:
        # Generujemy różne kombinacje wyszukiwania
        search_variations = [
            f"{keyword}",
            f"top {keyword}",
            f"best {keyword}",
            f"{keyword} 2024",
            f"{keyword} android",
            f"{keyword} app",
            f"{keyword} popular"
        ]
        
        for search_query in search_variations:
            search_url = f"https://play.google.com/store/search?q={search_query}&c=apps"
            print(f"\nPrzeszukiwanie: {search_url}")
            driver.get(search_url)
            time.sleep(3)
            
            scroll_count = 0
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while scroll_count < 100:  # Zwiększamy liczbę przewinięć
                # Przewijanie strony
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1)
                
                # Pobieranie elementów aplikacji
                app_elements = driver.find_elements(By.CSS_SELECTOR, "a[href*='/store/apps/details']")
                
                for app_element in app_elements:
                    try:
                        app_url = app_element.get_attribute('href')
                        app_id = app_url.split('id=')[-1].split('&')[0]
                        
                        if not any(app['app_id'] == app_id for app in apps):
                            title = app_element.get_attribute('aria-label')
                            
                            try:
                                developer = app_element.find_element(By.CSS_SELECTOR, "[class*='developer']").text
                            except:
                                developer = "Nieznany"
                                
                            try:
                                rating = app_element.find_element(By.CSS_SELECTOR, "[class*='rating']").text
                            except:
                                rating = "Brak ocen"
                            
                            app_info = {
                                'app_id': app_id,
                                'title': title,
                                'developer': developer,
                                'rating': rating,
                                'url': app_url,
                                'search_query': search_query
                            }
                            
                            apps.append(app_info)
                            if len(apps) % 100 == 0:
                                print(f"Znaleziono {len(apps)} aplikacji...")
                            
                    except Exception as e:
                        continue
                
                # Sprawdź czy strona się przewinęła
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
                scroll_count += 1
                
            print(f"Zakończono przeszukiwanie dla '{search_query}' - znaleziono {len(apps)} aplikacji")
            
        return apps
        
    except Exception as e:
        print(f"Błąd podczas wyszukiwania '{keyword}': {str(e)}")
        return apps  # Zwracamy co znaleźliśmy, nawet w przypadku błędu

def save_to_json(data, filename=None):
    """Zapisuje dane do pliku JSON"""
    try:
        if filename is None:
            filename = f"apps_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        
        # Użyj aktualnego katalogu
        current_dir = os.getcwd()
        filepath = os.path.join(current_dir, filename)
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        
        print(f"\nDane zapisano do pliku:")
        print(f"- Nazwa pliku: {filename}")
        print(f"- Pełna ścieżka: {filepath}")
        print(f"- Rozmiar pliku: {os.path.getsize(filepath)} bajtów")
        
    except Exception as e:
        print(f"Błąd podczas zapisywania pliku: {str(e)}")

def main():
    # Rozszerzona lista słów kluczowych bazująca na strukturze Google Play
    keywords = []
    
    # Dodajemy słowa kluczowe na podstawie PLAY_STORE_STRUCTURE
    for section in PLAY_STORE_STRUCTURE.values():
        for category_id, category_name in section['categories'].items():
            # Dodajemy angielskie nazwy kategorii
            keywords.append(category_id.lower().replace('_', ' '))
            # Dodajemy polskie nazwy kategorii
            keywords.append(category_name.lower())
            
    # Dodajemy popularne frazy
    additional_keywords = [
        "popular", "top", "best", "new", "trending",
        "essential", "must have", "recommended", "featured",
        "popularne", "najlepsze", "nowe", "polecane",
        "darmowe", "free", "premium", "pro",
        "2024", "android", "mobile", "phone"
    ]
    
    keywords.extend(additional_keywords)
    
    driver = setup_driver()
    all_apps = {}
    batch_count = 0
    batch_size = 10000  # Zapisujemy co 10000 aplikacji
    
    try:
        for keyword in keywords:
            print(f"\nWyszukiwanie dla frazy: {keyword}")
            apps = search_apps_by_keyword(driver, keyword, max_apps=5000)
            
            # Dodaj znalezione aplikacje do głównego słownika
            for app in apps:
                all_apps[app['app_id']] = app
            
            print(f"Łącznie znaleziono {len(all_apps)} unikalnych aplikacji")
            
            # Zapisuj wyniki co określoną liczbę aplikacji
            if len(all_apps) >= (batch_count + 1) * batch_size:
                batch_count += 1
                batch_filename = f"apps_data_batch_{batch_count}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
                save_to_json(list(all_apps.values()), batch_filename)
            
            time.sleep(2)
            
    except KeyboardInterrupt:
        print("\nPrzerwano wyszukiwanie przez użytkownika")
    finally:
        driver.quit()
        
    # Zapisz końcowe wyniki
    final_apps = list(all_apps.values())
    print(f"\nZnaleziono łącznie {len(final_apps)} unikalnych aplikacji")
    save_to_json(final_apps)

PLAY_STORE_STRUCTURE = {
    "APPLICATIONS": {
        "name": "Aplikacje",
        "categories": {
            "COMMUNICATION": "Komunikacja",
            "SOCIAL": "Społecznościowe",
            "PRODUCTIVITY": "Produktywność",
            "ENTERTAINMENT": "Rozrywka",
            "EDUCATION": "Edukacja",
            "BUSINESS": "Biznes",
            "FINANCE": "Finanse",
            "HEALTH_AND_FITNESS": "Zdrowie i fitness",
            "LIFESTYLE": "Styl życia",
            "MUSIC_AND_AUDIO": "Muzyka i dźwięk",
            "NEWS_AND_MAGAZINES": "Wiadomości i czasopisma",
            "PHOTOGRAPHY": "Fotografia",
            "SHOPPING": "Zakupy",
            "TOOLS": "Narzędzia",
            "TRAVEL_AND_LOCAL": "Podróże i lokalne",
            "VIDEO_PLAYERS": "Odtwarzacze wideo",
            "WEATHER": "Pogoda"
        }
    },
    "GAMES": {
        "name": "Gry",
        "categories": {
            "ACTION": "Akcja",
            "ADVENTURE": "Przygodowe",
            "ARCADE": "Zręcznościowe",
            "BOARD": "Planszowe",
            "CARD": "Karciane",
            "CASINO": "Kasyno",
            "CASUAL": "Casual",
            "EDUCATIONAL": "Edukacyjne",
            "MUSIC": "Muzyczne",
            "PUZZLE": "Logiczne",
            "RACING": "Wyścigi",
            "ROLE_PLAYING": "RPG",
            "SIMULATION": "Symulacje",
            "SPORTS": "Sportowe",
            "STRATEGY": "Strategiczne",
            "TRIVIA": "Quiz",
            "WORD": "Słowne"
        }
    },
    "FAMILY": {
        "name": "Dla rodziny",
        "categories": {
            "EDUCATION": "Edukacja",
            "MUSIC_AND_VIDEO": "Muzyka i wideo",
            "BRAIN_GAMES": "Gry logiczne",
            "CREATIVITY": "Kreatywność",
            "ACTION_AND_ADVENTURE": "Akcja i przygoda",
            "PRETEND_PLAY": "Zabawy",
            "AGE_5_AND_UNDER": "5 lat i młodsze",
            "AGE_6_8": "6-8 lat",
            "AGE_9_UP": "9 lat i starsze"
        }
    }
}

if __name__ == "__main__":
    main() 